In [17]:
import os 

In [25]:
%pwd

'c:\\Users\\pavan\\Desktop\\Fraud-Detection-E2E-ML-project-using-MLflow\\Notebooks'

In [26]:
os.chdir('../')

In [2]:
import boto3

In [6]:
%pwd

'c:\\Users\\pavan\\Desktop\\Fraud-Detection-E2E-ML-project-using-MLflow'

In [5]:
import os
os.chdir('../')

In [21]:
from research.download import download_file
from pathlib import Path

ImportError: attempted relative import with no known parent package

In [15]:
data = download_file('s3bucketfrauddata', 'fraud_data.csv')

NameError: name 'download_file' is not defined

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [28]:
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import read_yaml, create_directories

In [29]:
class ConfigurationManager:
    # First setting up the environment by defining the file paths of configuration file, params file and schema file.  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Creating the root directory in the artifacts folder
        create_directories([self.config.artifacts_root])


    # Data Ingestion step
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            # We are accessing the file paths that are defined in the configuration yaml file
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [30]:
import os
import urllib.request as request
from fraud_detection_project import logger
from fraud_detection_project.utils.common import get_size

In [31]:
# So far, We have configured the structure of the data ingestion
# Now to execute the steps like downloading from the source url lets define the function to peform those actions

# initializing the data ingestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # defining the function to download the csv file from the source url
    def download_file(self):
        # checks if the data file is already present or not, if not executes the function
        if not os.path.exists(self.config.local_data_file):
            # Extracting the information of the source URL like dataset file name and url headers
            filename, headers = request.urlretrieve(
                # we are retriving the url path and file name that is defined in the config file
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


In [32]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-08-22 10:48:51,289: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-22 10:48:51,313: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-22 10:48:51,313: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-22 10:48:51,321: INFO: common: created directory at: artifacts]
[2023-08-22 10:48:51,331: INFO: common: created directory at: artifacts/data_ingestion]


[2023-08-22 10:48:52,005: INFO: 233940858: artifacts/data_ingestion/fraud_data.csv downloaded with following info: 
Content-Type: text/html; charset=utf-8
Date: Tue, 22 Aug 2023 14:48:51 GMT
Cache-Control: no-cache, no-store
Pragma: no-cache
Set-Cookie: ka_sessionid=d6105c0cae3f8626a5f39258f0bd36e0; max-age=2626560; path=/
Set-Cookie: CSRF-TOKEN=CfDJ8OUZZhoRU_5EmXzVc6iDdFUqWvTIkhFOhzUxHpP9jL0lvZcHCzgmHidVBclxzqiuPKzHfszueEL99POGeIP6DkKBhrfx0UjtLxXq-viNcg; path=/; secure; samesite=lax; httponly
Set-Cookie: XSRF-TOKEN=CfDJ8OUZZhoRU_5EmXzVc6iDdFWub2-5lCCGlTzaRYWX5escsRmoC0277yik1bqAK21x451sDTVjjWZ0mNAmxXv9MF-48qe39Zrz7oFDxMYYj6lmxw; path=/; secure; samesite=lax
Set-Cookie: CLIENT-TOKEN=eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJrYWdnbGUiLCJhdWQiOiJjbGllbnQiLCJzdWIiOm51bGwsIm5idCI6IjIwMjMtMDgtMjJUMTQ6NDg6NTIuMTc4NjcyOFoiLCJpYXQiOiIyMDIzLTA4LTIyVDE0OjQ4OjUyLjE3ODY3MjhaIiwianRpIjoiOTc5YTFjYmYtZWY0Zi00ZTQyLWE1NjctMDFlMjIyNGFlNDAzIiwiZXhwIjoiMjAyMy0wOS0yMlQxNDo0ODo1Mi4xNzg2NzI4WiIsImFub24i

In [24]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')